In [1]:
!pip install mp-api --upgrade
import numpy as np
#from pymatgen.ext.matproj import MPRester
from mp_api.client import MPRester
from pymatgen.core.operations import SymmOp
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.electronic_structure.plotter import BSPlotter
from pymatgen.phonon.plotter import PhononBSPlotter
from jupyter_jsmol.pymatgen import quick_view
from lmapr1492 import plot_brillouin_zone, get_plot_bs, get_plot_dos, get_plot_bs_and_dos, get_branch_wavevectors
from plotly.subplots import make_subplots
import plotly.graph_objects as go

Defaulting to user installation because normal site-packages is not writeable


In [2]:
mp_key = "x6hAWrx1l7nO3pcKMHzrj9I3u9zQTpCo"
mp_id = "mp-924130"

In [4]:
m = MPRester(mp_key)
struct = m.get_structure_by_material_id(mp_id, conventional_unit_cell=True)
print(struct)
# Coordonnées de 3 atomes dans la maille conventionelle
Ti = [0, 0, 0]
Ni = [0.25, 0.25, 0.25]
Sn = [0.5, 0, 0]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Full Formula (Ti4 Ni4 Sn4)
Reduced Formula: TiNiSn
abc   :   5.906882   5.906882   5.906882
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (12)
  #  SP       a     b     c
---  ----  ----  ----  ----
  0  Ti    0.5   0.5   0
  1  Ti    0.5   0     0.5
  2  Ti    0     0.5   0.5
  3  Ti    0     0     0
  4  Ni    0.25  0.25  0.25
  5  Ni    0.25  0.75  0.75
  6  Ni    0.75  0.25  0.75
  7  Ni    0.75  0.75  0.25
  8  Sn    0     0.5   0
  9  Sn    0     0     0.5
 10  Sn    0.5   0.5   0.5
 11  Sn    0.5   0     0


In [62]:
# Je n'arrive vraiment pas à faire fonctionner quick_view

view.script()

JsmolView(layout=Layout(align_self='stretch', height='400px'))

In [5]:
sga = SpacegroupAnalyzer(struct)
# Toutes les opérations de symétrie possibles
sym = sga.get_symmetry_operations()
# J'en sélectionne 3 en étant sûr de rejeter l'opération identité
sym1 = sym[1] 
sym2 = sym[4]
sym3 = sym[2]

Simples calculs matriciels résolues sur papier pour trouver les [x, y , z] après l'application de l'opération

In [6]:
# Symétrie 1
print(f"Première opération de symétrie")
print("==================================================")
print(f"Opération :\n {sym1}")
print(f"Atome étudié : Ni")
print(f"Position avant l'opération : {Ni}")
Ni_sym = sym1.operate(Ni)
print(f"Position après l'opération : {Ni_sym}")
print(f"Cette opération est une rotation de -90° (sens trigonométrique = sens positif) selon l'axe z (c) ([x, y, z]  ==>  [y, -x, z])")
view = quick_view(struct)
display(view)
view.script("draw symop 2 {atomno = 5}")


Première opération de symétrie
Opération :
 Rot:
[[ 0.  1.  0.]
 [-1.  0.  0.]
 [ 0.  0. -1.]]
tau
[0. 0. 0.]
Atome étudié : Ni
Position avant l'opération : [0.25, 0.25, 0.25]
Position après l'opération : [ 0.25 -0.25 -0.25]
Cette opération est une rotation de -90° (sens trigonométrique = sens positif) selon l'axe z ([x, y, z]  ==>  [y, -x, z])


JsmolView(layout=Layout(align_self='stretch', height='400px'))

In [9]:
# Symétrie 2
print(f"\n\nSeconde opération de symétrie")
print("==================================================")
print(f"Opération :\n {sym2}")
print(f"Atome étudié : Ti")
print(f"Position avant l'opération : {Ti}")
Ti_sym = sym2.operate(Ti)
print(f"Position après l'opération : {Ti_sym}")
print(f"Cette opération est une symétrie axiale selon l'axe x (a) ([x, y, z]  ==>  [x, -y, -z])")
view = quick_view(struct)
display(view)
view.script("draw symop 5 {atomno = 4}")



Seconde opération de symétrie
Opération :
 Rot:
[[ 1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]]
tau
[0. 0. 0.]
Atome étudié : Ti
Position avant l'opération : [0, 0, 0]
Position après l'opération : [0. 0. 0.]
Cette opération est une symétrie axiale selon l'axe x (a) ([x, y, z]  ==>  [x, -y, -z])


JsmolView(layout=Layout(align_self='stretch', height='400px'))

In [10]:
# Symétrie 3
print(f"\n\nTroisième opération de symétrie")
print("==================================================")
print(f"Opération :\n {sym3}")
print(f"Atome étudié : Sn")
print(f"Position avant l'opération : {Sn}")
Sn_sym = sym3.operate(Sn)
print(f"Position après l'opération : {Sn_sym}")
print(f"Cette opération est une symétrie axiale selon l'axe z (c) ([x, y, z]  ==>  [-x, -y, z])")
view = quick_view(struct)
display(view)
view.script("draw symop 3 {atomno = 12}")



Troisième opération de symétrie
Opération :
 Rot:
[[-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0.  1.]]
tau
[0. 0. 0.]
Atome étudié : Sn
Position avant l'opération : [0.5, 0, 0]
Position après l'opération : [-0.5  0.   0. ]
Cette opération est une symétrie axiale selon l'axe z () ([x, y, z]  ==>  [-x, -y, z])


JsmolView(layout=Layout(align_self='stretch', height='400px'))